In [198]:
import os
import re
import numpy as np
import pandas as pd
from datetime import datetime

In [199]:
# Ruta del archivo csv 
FILES_PATH = os.path.join('..','..','data','raw','GeekyMotors')
csv_path = os.path.join(FILES_PATH, 'geeky_motors2.csv')

# Importación del archivo CSV
data = pd.read_csv(csv_path, encoding="latin-1")
data

,Nombre,KM,Estado,Combustible,Carrocería,Tracción,Transmisión,Asientos,Color,Verificación,Precio
0,Tesla Model 3 2018 Bajo Perfil,44000,Usado,Eléctrico,Sedan,4x4,Automática,Cuero vegano,Red Multi-Coat,93.0,"$55,000"
1,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
2,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
3,Toyota Corolla Sport Edition Full Extras 2019,78000,Usado,Gasolina,Sedan,4x2,Automática,Cuero,Azul Marino,95.0,"$17,500"
4,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
...,...,...,...,...,...,...,...,...,...,...,...
4283,Nissan Almera 2005,150000,Usado,Gasolina,Sedan,4x2,Automática,Tela,Rojo Vino,NaN,"$3,500"
4284,Kia Sportage full xtra 2017 Negociable,96000,Usado,Gasolina,SUV,4x2,Automática,"Tela, oscuros y áreas de cuero",Negro / gris,NaN,"$12,999"
4285,BMW X5 3.OD 2014,167000,Usado,Diesel,SUV,4x4,Automática,cuero,crema,NaN,"$18,000"
4286,Jeep Wrangler Unlimited 2018,78000,Usado,Gasolina,SUV,4x4,Automática,Tela,Beige,NaN,"$25,000"


In [200]:
for col in data.columns:
    if data[col].nunique() < 20:
        print(col)
        print(data[col].unique())
        print()

Estado
['Usado']

Combustible
['Eléctrico' 'Gasolina' 'Diesel']

Carrocería
['Sedan' 'SUV' 'Pickup' 'Coupe' 'Hatchback' 'Convertible' 'Wagon']

Tracción
['4x4' '4x2']

Transmisión
['Automática' 'Manual']

Asientos
['Cuero vegano' 'Cuerro' 'Cuero' 'Algodón' 'Tela' 'cuero suave' 'cuero'
 'CUERO' 'tela' 'Asientos de tela con forros de cuerinq' 'Tela, off road'
 'Cuerina Premium' 'TELA' 'Cuero rojo con negro' 'Asientos de tela' '5'
 'Cuerina' 'Tela, oscuros y áreas de cuero']

Verificación
[ 93.  nan  95.  92. 100.  98.]



In [201]:
data_tmp = pd.DataFrame()

### Marca

In [202]:
def estandarizar_marcas(col, reemplazos_marcas):
    """
    Esta función estandariza los nombres de marcas en una columna de un DataFrame de pandas
    según un diccionario de reemplazos proporcionado. Los nombres de marcas en la columna se
    reemplazan con claves específicas del diccionario si se encuentran las variantes especificadas.

    Parameters:
    - col: pd.Series
        La columna de un dataframe que contiene los nombres de marca a estandarizar.
    - reemplazos_marcas: dict
        Un diccionario donde las claves son las marcas estandarizadas y los valores son listas
        de variantes de esas marcas que deben ser reemplazadas por la clave correspondiente.

    Retorna:
    - pd.Series
        la columna con las marcas standarizadas.
    """

    # Iterar sobre el diccionario de reemplazos donde la clave es la marca
    # y los valores son las marcas digitadas con errores
    for clave, valores in reemplazos_marcas.items():
        for valor in valores:
            # Reemplaza cada variante en la columna con la marca estandarizada
            # `valor.lower()` asegura que el reemplazo no distinga entre mayúsculas y minúsculas
            # `case=False` permite que el reemplazo sea insensible a mayúsculas y minúsculas
            # `regex=False` desactiva el uso de expresiones regulares para el reemplazo
            col = col.str.replace(rf'{valor.lower()}', clave, case=False, regex=False)
            

    return col

In [203]:
reemplazos_marcas = {'Hyundai':['hyudani'], 'Porsche':['porche'], 'Mercedes-Benz':['mercedes']}
data_tmp['nombre'] = estandarizar_marcas(data.Nombre, reemplazos_marcas)
data_tmp.nombre

0                      Tesla Model 3 2018 Bajo Perfil
1                                        Audi Q5 2012
2                                        Audi Q5 2012
3       Toyota Corolla Sport Edition Full Extras 2019
4                                        Audi Q5 2012
                            ...                      
4283                               Nissan Almera 2005
4284           Kia Sportage full xtra 2017 Negociable
4285                                 BMW X5 3.OD 2014
4286                     Jeep Wrangler Unlimited 2018
4287                                TOYOTA HILUX 2019
Name: nombre, Length: 4288, dtype: object

In [204]:
# Lista de marcas de autos
lista_marcas = [
    "acura", "alfa romeo", "aston martin", "audi", "bentley", "bmw", "bugatti", "buick",
    "cadillac", "chevrolet", "chrysler", "citroën", "dacia", "daewoo", "daihatsu", "dodge",
    "ferrari", "fiat", "ford", "geely", "genesis", "gmc", "honda", "hummer", "hyundai", "infiniti",
    "isuzu", "jaguar", "jeep", "kia", "koenigsegg", "lamborghini", "lancia", "land rover",
    "lexus", "lincoln", "lotus", "mahindra", "maserati", "maybach", "mazda", "mclaren", "mercedes-benz",
    "mercury", "mg", "mini", "mitsubishi", "morgan", "nissan", "opel", "pagani", "peugeot",
    "plymouth", "polestar", "pontiac", "porsche", "ram", "renault", "rolls-royce", "saab",
    "seat", "škoda", "smart", "subaru", "suzuki", "tata", "tesla", "toyota", "volkswagen",
    "volvo", "wiesmann", "zotye", "byd", "chery", "jac", "ssangyong", "great wall", "fisker", "rivian", "lucid", "vinfast"
]


In [205]:
def extraer_marca(df):
    # Convertir el texto a minúsculas
    texto = df.lower()

    for marca in lista_marcas:
        # Crear un patrón de expresión regular para la marca, con delimitadores de palabra
        patron = r'\b' + re.escape(marca.lower()) + r'\b'

        # Si encuentra una coincidencia de palabra completa, retorna la marca
        if re.search(patron, texto):
            return marca

    return df 

In [206]:
data_tmp['marca'] = data_tmp['nombre'].apply(extraer_marca)
data_tmp.marca.unique()

array(['tesla', 'audi', 'toyota', 'nissan', 'hyundai', 'kia', 'bmw',
       'honda', 'mini', 'mercedes-benz', 'subaru', 'suzuki', 'renault',
       'mazda', 'acura', 'geely', 'lincoln', 'land rover', 'ford',
       'maserati', 'mitsubishi', 'mahindra', 'peugeot', 'isuzu',
       'volkswagen', 'chevrolet', 'jeep', 'dodge', 'porsche'],
      dtype=object)

### Año

In [207]:
def extraer_anio(df):
    lista_anios = np.arange(1990, datetime.now().year + 1)
    for anio in lista_anios:
        if str(anio) in df:
            return anio
    return ""

In [208]:

data_tmp['anio'] = data['Nombre'].apply(extraer_anio)
data_tmp['anio']

0       2018
1       2012
2       2012
3       2019
4       2012
        ... 
4283    2005
4284    2017
4285    2014
4286    2018
4287    2019
Name: anio, Length: 4288, dtype: int64

### Modelo

In [209]:
def extraer_modelos(df):
    marca = extraer_marca(df)
    anio = extraer_anio(df)

    if marca != "Desconocida" and anio != "Desconocido":
        # Encontrar la posición de la marca y el año en el texto
        start_index = df.lower().find(marca.lower()) + len(marca)
        end_index = df.find(str(anio))

        # Extraer el texto entre la marca y el año
        modelo = df[start_index:end_index].strip()

        return modelo
    return ""

In [210]:
data_tmp['modelo'] = data['Nombre'].apply(extraer_modelos)
data_tmp['modelo'].unique()

array(['Model 3', 'Q5', 'Corolla Sport Edition Full Extras', 'Navara',
       'i20-Active', 'Extrail', 'RIO', 'Cerato', '218i Gran Coupé',
       'Civic full extra', 'Cooper', 'Mercedes AMG GLE 63s Coupe', 'XV',
       'Ciaz', 'Tucson', 'Model X', 'Xtrail', 'Captur', '318i', 'Crv',
       'Q3', 'Accent', 'Corolla', 'X-trail', 'CX-9', 'Tucson GLS Full',
       'Land Cruiser', 'Rogue', 'TLX', 'GLE 400', 'Grand i10', '650i', '',
       'LAND CRUISER Prado', 'EON', 'Civic', 'Swift dzire', 'Coolray',
       'Outback', 'K2700', 'Aviator', 'CR-V', 'Sportage Gt Line',
       'Tundra', 'Rav 4', 'Yaris', 'MDX',
       'Range Rover Vogue supercharged', 'Range Rover Sport', 'Cx5',
       'tercel', 'A3', 'F150 Lariat', 'Kicks', 'Venue', 'X1',
       'Levante S Negociable', 'HILUX', 'F-150 Raptor', '640',
       'RIO HATCHBACK', 'Yaris S FULL', 'Asx', 'Vitara', 'Figo',
       'Q3 Sportback', 'Ranger XL', 'BRONCO SPORT', 'COROLLA SE',
       'Pik Up 2.5', '301', 'ILX', 'ISUZU', 'TIGUAN', 'Traverse',


In [211]:
def remover_textos(col, textos_a_eliminar):
    """
    Elimina palabras como 'full extra', 'full extras', 'hatchback', etc de la columna "modelo",
    sin importar si están en mayúsculas o minúsculas.
    """
    pattern = re.compile(r'\b(?:' + '|'.join(re.escape(texto) for texto in textos_a_eliminar) + r')\b', re.IGNORECASE)
    modelo_tmp = col.apply(lambda x: pattern.sub('', str(x)).strip())

    return modelo_tmp

In [212]:
# Lista de textos a eliminar
textos_a_eliminar = ['full', 'xtra', 'extra', 'extras', 'full extra', 'full extras', 'full xtra', 'hatchback', 'negociable', 'mercedes']
data_tmp['modelo'] = remover_textos(data_tmp.modelo, textos_a_eliminar)
data_tmp.modelo.unique()

array(['Model 3', 'Q5', 'Corolla Sport Edition', 'Navara', 'i20-Active',
       'Extrail', 'RIO', 'Cerato', '218i Gran Coupé', 'Civic', 'Cooper',
       'AMG GLE 63s Coupe', 'XV', 'Ciaz', 'Tucson', 'Model X', 'Xtrail',
       'Captur', '318i', 'Crv', 'Q3', 'Accent', 'Corolla', 'X-trail',
       'CX-9', 'Tucson GLS', 'Land Cruiser', 'Rogue', 'TLX', 'GLE 400',
       'Grand i10', '650i', '', 'LAND CRUISER Prado', 'EON',
       'Swift dzire', 'Coolray', 'Outback', 'K2700', 'Aviator', 'CR-V',
       'Sportage Gt Line', 'Tundra', 'Rav 4', 'Yaris', 'MDX',
       'Range Rover Vogue supercharged', 'Range Rover Sport', 'Cx5',
       'tercel', 'A3', 'F150 Lariat', 'Kicks', 'Venue', 'X1', 'Levante S',
       'HILUX', 'F-150 Raptor', '640', 'Yaris S', 'Asx', 'Vitara', 'Figo',
       'Q3 Sportback', 'Ranger XL', 'BRONCO SPORT', 'COROLLA SE',
       'Pik Up 2.5', '301', 'ILX', 'ISUZU', 'TIGUAN', 'Traverse',
       'Grand Cherokee Laredo', 'Ford EDGE SEL', 'Cx-5', 'WRANGLER',
       'Range Rover Evoq

In [213]:
def transformar_sport_edition(col):
    """
    Transforma el texto "Sport Edition" a "SE"
    """

    pattern = re.compile(r'\bsport[\s-]*edition\b', re.IGNORECASE)
    modelo_tmp = col.apply(lambda x: pattern.sub('SE', str(x)))

    return modelo_tmp

In [214]:
data_tmp['modelo'] = transformar_sport_edition(data_tmp.modelo)
print(f"Cantidad de modelos: {data_tmp.modelo.nunique()}")
data_tmp.modelo.unique()

Cantidad de modelos: 96


array(['Model 3', 'Q5', 'Corolla SE', 'Navara', 'i20-Active', 'Extrail',
       'RIO', 'Cerato', '218i Gran Coupé', 'Civic', 'Cooper',
       'AMG GLE 63s Coupe', 'XV', 'Ciaz', 'Tucson', 'Model X', 'Xtrail',
       'Captur', '318i', 'Crv', 'Q3', 'Accent', 'Corolla', 'X-trail',
       'CX-9', 'Tucson GLS', 'Land Cruiser', 'Rogue', 'TLX', 'GLE 400',
       'Grand i10', '650i', '', 'LAND CRUISER Prado', 'EON',
       'Swift dzire', 'Coolray', 'Outback', 'K2700', 'Aviator', 'CR-V',
       'Sportage Gt Line', 'Tundra', 'Rav 4', 'Yaris', 'MDX',
       'Range Rover Vogue supercharged', 'Range Rover Sport', 'Cx5',
       'tercel', 'A3', 'F150 Lariat', 'Kicks', 'Venue', 'X1', 'Levante S',
       'HILUX', 'F-150 Raptor', '640', 'Yaris S', 'Asx', 'Vitara', 'Figo',
       'Q3 Sportback', 'Ranger XL', 'BRONCO SPORT', 'COROLLA SE',
       'Pik Up 2.5', '301', 'ILX', 'ISUZU', 'TIGUAN', 'Traverse',
       'Grand Cherokee Laredo', 'Ford EDGE SEL', 'Cx-5', 'WRANGLER',
       'Range Rover Evoque', '4Runn

In [215]:
def estandarizar_modelos(col, dict_reemplazos):
    # Iterar sobre el diccionario de reemplazos
    for clave, valores in dict_reemplazos.items():
        for valor in valores:
            # Reemplaza cada valor en la columna temporal en minúsculas
            col = col.str.replace(rf'{valor}', clave, case=False, regex=False)

    return col


In [216]:
reemplazos = {'X-Trail': ['extrail','xtrail','X-trail'], 'CRV':['Crv','cr-v'], 'CX5':['Cx-5','cx5'], 'Rio':['RIO'],
              'Pick Up':['pik up']}
data_tmp['modelo']=estandarizar_modelos(data_tmp.modelo, reemplazos)
print(f"Cantidad de modelos: {data_tmp.modelo.nunique()}")
data_tmp.modelo.unique()

Cantidad de modelos: 92


array(['Model 3', 'Q5', 'Corolla SE', 'Navara', 'i20-Active', 'X-Trail',
       'Rio', 'Cerato', '218i Gran Coupé', 'Civic', 'Cooper',
       'AMG GLE 63s Coupe', 'XV', 'Ciaz', 'Tucson', 'Model X', 'Captur',
       '318i', 'CRV', 'Q3', 'Accent', 'Corolla', 'CX-9', 'Tucson GLS',
       'Land Cruiser', 'Rogue', 'TLX', 'GLE 400', 'Grand i10', '650i', '',
       'LAND CRUISER Prado', 'EON', 'Swift dzire', 'Coolray', 'Outback',
       'K2700', 'Aviator', 'Sportage Gt Line', 'Tundra', 'Rav 4', 'Yaris',
       'MDX', 'Range Rover Vogue supercharged', 'Range Rover Sport',
       'CX5', 'tercel', 'A3', 'F150 Lariat', 'Kicks', 'Venue', 'X1',
       'Levante S', 'HILUX', 'F-150 Raptor', '640', 'Yaris S', 'Asx',
       'Vitara', 'Figo', 'Q3 Sportback', 'Ranger XL', 'BRONCO SPORT',
       'COROLLA SE', 'Pick Up 2.5', '301', 'ILX', 'ISUZU', 'TIGUAN',
       'Traverse', 'Grand Cherokee Laredo', 'Ford EDGE SEL', 'WRANGLER',
       'Range Rover Evoque', '4Runner', 'WRX', 'Prado TXL',
       'Outlander 

### Precio

In [217]:
data['Precio'] = data['Precio'].str.replace('$','').str.replace(',','')
data['Precio'] = data['Precio'].astype(int)


In [218]:
data_tmp.drop('nombre', axis=1, inplace=True)
data_tmp['carroceria'] = data.Carrocería.copy()
data_tmp['traccion'] = data.Tracción.copy()
data_tmp['transmision'] = data.Transmisión.copy()
data_tmp['combustible'] = data.Combustible.copy()
data_tmp['kilometraje'] = data.KM.copy()
data_tmp['verificacion'] = data.Verificación.copy()
data_tmp['Precio'] = data.Precio.copy()
data_tmp

,marca,anio,modelo,carroceria,traccion,transmision,combustible,kilometraje,verificacion,Precio
0,tesla,2018,Model 3,Sedan,4x4,Automática,Eléctrico,44000,93.0,55000
1,audi,2012,Q5,SUV,4x2,Automática,Gasolina,154000,NaN,6500
2,audi,2012,Q5,SUV,4x2,Automática,Gasolina,154000,NaN,6500
3,toyota,2019,Corolla SE,Sedan,4x2,Automática,Gasolina,78000,95.0,17500
4,audi,2012,Q5,SUV,4x2,Automática,Gasolina,154000,NaN,6500
...,...,...,...,...,...,...,...,...,...,...
4283,nissan,2005,Almera,Sedan,4x2,Automática,Gasolina,150000,NaN,3500
4284,kia,2017,Sportage,SUV,4x2,Automática,Gasolina,96000,NaN,12999
4285,bmw,2014,X5 3.OD,SUV,4x4,Automática,Diesel,167000,NaN,18000
4286,jeep,2018,Wrangler Unlimited,SUV,4x4,Automática,Gasolina,78000,NaN,25000


In [219]:
data_tmp.dtypes

marca            object
anio              int64
modelo           object
carroceria       object
traccion         object
transmision      object
combustible      object
kilometraje       int64
verificacion    float64
Precio            int64
dtype: object